# stf-decomposition demo
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thodson-usgs/stf-decomposition/blob/main/notebooks/stf-decomposition-demo.ipynb)  
This notebook demonstrates STF: seasonal-seasonal trend decomposition using the Fast Fourier transform.

TODO add short summary

In [ ]:
%%capture
# Add package to Colab environment
#!pip install stf-decomposition
!pip install git+https://github.com/thodson-usgs/stf-decomposition.git
!pip install intake requests aiohttp
!pip install statsmodels #CoLab version is outdated

In [ ]:
# load the data catalog
import intake
url = 'https://raw.githubusercontent.com/thodson-usgs/stf-decomposition/main/data/stf_data_catalog.yml'
cat = intake.open_catalog(url)
list(cat)

In [ ]:
# load a dataset
df = cat['co2'].read()

df.head()

## Basic usage
TODO add short summary

In [ ]:
# TODO demonstrate basic usage on co2 dataset
from stf_decomposition import STF

## Comparison with STL
The functionality of stf_decomposition was modeled after the 
[statsmodels.tsa.seasonal.STL](https://www.statsmodels.org/devel/examples/notebooks/generated/stl_decomposition.html) library. 
The motivation in modeling stf_decomposition after STL was to create a decomposition framework
that is already familiar to useres for ease of use. 
In the code below we can observe that the visually the decomposed components after STL and stf_decoposition
are very similar in this example. Two different stf_decomposition call were run in order to see
the difference between a user input seasonal variable and the optimized seasonal variable.


In [ ]:
import statsmodels
statsmodels.__version__

In [ ]:
# TODO make a multi-panel that compares STF and STL on several datasets
# you can add more datasets to the data/stf_data_catalog.yml either through our github repo or any URL
from stf_decomposition import STF
from statsmodels.tsa.seasonal import STL

retail = cat['retail_sales'].read()

stl = STL(retail, seasonal = 13)
res = stl.fit()
fig = res.plot()

# Perform STF with user input window 
stf = STF(retail, "blackman", seasonal = 13)
res = stf.fit()
fig = res.plot()

# Perform STF with optimized window
stf = STF(retail, "blackman")
res = stf.fit()
fig = res.plot()

## Window Optimization
Seasonal window optimization is performed if a user does not give a seasonal input. 
The optimization method used in seasonal window selection is 
[scipy.optimize.brute] (https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brute.html).
This optimization method was chosen as it returns the global minimum 
of a given function over a given range of values, in this case a range of 3 to 100.


In [ ]:
# Window Optimization
co2 = cat['co2'].read()

stf = STF(co2, "blackman")
res = stf.fit()
fig = res.plot()

print(stf.seasonal)

In [ ]:
air = cat['air_passengers'].read()

stf = STF(air, "blackman")
res = stf.fit()
fig = res.plot()

print(stf.seasonal)